In [2]:
import numpy as np
import scipy.optimize as opt

In [8]:
def f1 (X):
    X = np.array(x)
    C = [0.06, 0.53, 0.18, 0.18, 0.06]
    return X.dot(C)

def f2 (X):
    X = np.array(x)
    C = [25, 70, 60, 95, 45]
    return X.dot(C)

def f3 (X):
    X = np.array(x)
    C = [0, 32.5, 300, 120, 0]
    return X.dot(C)

def f4 (X):
    X = np.array(x)
    C = [0.1, 0.1, 0.11, 0.35, 0.33]
    return X.dot(C)

In [11]:
linear_constraint = opt.LinearConstraint([1, 1, 1, 1, 1], 0, 3000)
bounds = opt.Bounds([0, 0, 0, 0, 0], [850, 220, 1300, 1615, 700])

In [16]:
opt.minimize(f1, [0, 0, 0, 0, 0], method="Powell", bounds=bounds, constraints=linear_constraint)

/Users/feliperamalho/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:518: RuntimeWarning: Method Powell cannot handle constraints nor bounds.
  RuntimeWarning)


   direc: array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])
     fun: 1.01
 message: 'Optimization terminated successfully.'
    nfev: 61
     nit: 1
  status: 0
 success: True
       x: array([2.58792896, 2.58792896, 2.58792896, 2.58792896, 2.58792896])